In [58]:
import pandas as pd
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV, Lasso, LinearRegression
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

In [59]:
ghg_change = pd.read_csv("../../data/mapped/mapped_cdp_ghg_change_processed.csv", index_col=['id', 'year'])

In [60]:
ghg_change = ghg_change.reset_index().drop(columns=['Unnamed: 0', 'id'])

In [61]:
ghg_change = ghg_change[ghg_change['absent_cdp_ghg_change_processed.csv'] == 0].drop(columns=['absent_cdp_ghg_change_processed.csv'])

In [62]:
ghg_change

,year,isin,ghg_change_total,ghg_change_real,ghg_change_structure,ghg_change_measure,ghg_change_othergroup,ghg_change_nonzero_count,ghg_change_real_cat,ghg_change_real_cat_next,ghg_change_real_next
0,2011.0,GB00B1YW4409,-11.00,-11.00,0.00,0.0,0.00,1.0,1.0,0.0,0.00
1,2012.0,GB00B1YW4409,-25.00,0.00,0.00,0.0,-25.00,1.0,0.0,1.0,-1.00
2,2013.0,GB00B1YW4409,-19.00,-1.00,0.00,-1.0,-17.00,3.0,1.0,0.0,0.00
3,2014.0,GB00B1YW4409,-29.00,0.00,0.00,0.0,-29.00,1.0,0.0,0.0,0.00
4,2016.0,GB00B1YW4409,-4.00,0.00,0.00,0.0,-4.00,1.0,0.0,1.0,-39.70
...,...,...,...,...,...,...,...,...,...,...,...
24032,2021.0,JP3323040000,-3.54,-3.11,0.00,0.0,-0.24,3.0,1.0,1.0,-2.18
24035,2021.0,DE0005800601,0.00,0.00,0.00,0.0,0.00,0.0,0.0,1.0,-29.60
24037,2021.0,EGS30901C010,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.00
24042,2021.0,BRVAMOACNOR7,7.19,0.00,7.19,0.0,0.00,1.0,0.0,1.0,-23.50


In [63]:
# ohe_ghg_change = pd.get_dummies(ghg_change, dtype=int)

In [99]:
# Create train and test sets
train_data = ghg_change[ghg_change['year'] < 2021].drop(columns=['ghg_change_real_cat_next'])
test_data = ghg_change[ghg_change['year'] == 2021].drop(columns=['ghg_change_real_cat_next'])

# # Define features and target variable
# target = 'ghg_change_real_next'

# X_train = train_data.drop(columns=['ghg_change_real_next', 'ghg_change_real_cat_next'])
# y_train = train_data[target]
# X_test = test_data.drop(columns=['ghg_change_real_next', 'ghg_change_real_cat_next'])
# y_test = test_data[target]


In [100]:
# drop isins in the test data that are not in the train data
train_isins = train_data['isin'].unique()
test_data = test_data[test_data['isin'].isin(train_isins)]

In [101]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split

# Assuming your DataFrame is named 'data'
# Replace 'y', 'firm_id', and 'year' with your actual column names
dependent_var = 'ghg_change_real_next'
non_feature_cols = ['isin', 'year', dependent_var]  # Include any non-feature columns

# Automatically construct a formula
features = '+'.join([f'Q("{col}")' if ' ' in col else col for col in train_data.columns if col not in non_feature_cols])
formula = f'{dependent_var} ~ {features} + C(isin) + year'



# Fit the model on training data
model = ols(formula, data=train_data).fit()

# Print model summary
print(model.summary())

# Predict on test data
test_data['predicted_y'] = model.predict(test_data)

# View the predictions
print(test_data[[dependent_var, 'predicted_y']])

                             OLS Regression Results                             
Dep. Variable:     ghg_change_real_next   R-squared:                       0.353
Model:                              OLS   Adj. R-squared:                  0.201
Method:                   Least Squares   F-statistic:                     2.333
Date:                  Mon, 11 Dec 2023   Prob (F-statistic):          4.10e-230
Time:                          15:10:22   Log-Likelihood:                -52183.
No. Observations:                 16213   AIC:                         1.105e+05
Df Residuals:                     13144   BIC:                         1.341e+05
Df Model:                          3068                                         
Covariance Type:              nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Inte

In [102]:
# calculate test rmse and r2
test_rmse = mean_squared_error(test_data[dependent_var], test_data['predicted_y'], squared=False)
test_r2 = r2_score(test_data[dependent_var], test_data['predicted_y'])

In [103]:
test_rmse

10.421244426814294

In [104]:
test_r2

-0.04342249991712532

In [97]:
train_data

,year,isin,ghg_change_total,ghg_change_real,ghg_change_structure,ghg_change_measure,ghg_change_othergroup,ghg_change_nonzero_count,ghg_change_real_cat,ghg_change_real_next,predicted_y,residuals
0,2011.0,GB00B1YW4409,-11.00,-11.00,0.0,0.00,0.00,1.0,1.0,0.00,-9.298019,-1.701981
1,2012.0,GB00B1YW4409,-25.00,0.00,0.0,0.00,-25.00,1.0,0.0,-1.00,-1.832773,1.832773
2,2013.0,GB00B1YW4409,-19.00,-1.00,0.0,-1.00,-17.00,3.0,1.0,0.00,-7.617360,6.617360
3,2014.0,GB00B1YW4409,-29.00,0.00,0.0,0.00,-29.00,1.0,0.0,0.00,-1.834498,1.834498
4,2016.0,GB00B1YW4409,-4.00,0.00,0.0,0.00,-4.00,1.0,0.0,-39.70,-1.823716,1.823716
...,...,...,...,...,...,...,...,...,...,...,...,...
23874,2020.0,TW0009940007,-6.28,-4.19,0.0,-2.09,0.00,3.0,1.0,-0.90,-5.677953,1.487953
23877,2020.0,JP3423800006,0.00,0.00,0.0,0.00,0.00,0.0,0.0,-1.29,-0.126095,0.126095
23880,2020.0,ZAE000090692,10.97,-0.38,0.0,0.00,4.07,3.0,1.0,-1.28,-4.197397,3.817397
23883,2020.0,GB00B4X1RC86,10.47,-0.03,0.0,1.50,0.00,3.0,1.0,-0.01,-3.972228,3.942228


In [69]:
train_data['isin'] = train_data['isin'].astype('category')

In [70]:
train_data

,year,isin,ghg_change_total,ghg_change_real,ghg_change_structure,ghg_change_measure,ghg_change_othergroup,ghg_change_nonzero_count,ghg_change_real_cat,ghg_change_real_next
0,2011.0,GB00B1YW4409,-11.00,-11.00,0.0,0.00,0.00,1.0,1.0,0.00
1,2012.0,GB00B1YW4409,-25.00,0.00,0.0,0.00,-25.00,1.0,0.0,-1.00
2,2013.0,GB00B1YW4409,-19.00,-1.00,0.0,-1.00,-17.00,3.0,1.0,0.00
3,2014.0,GB00B1YW4409,-29.00,0.00,0.0,0.00,-29.00,1.0,0.0,0.00
4,2016.0,GB00B1YW4409,-4.00,0.00,0.0,0.00,-4.00,1.0,0.0,-39.70
...,...,...,...,...,...,...,...,...,...,...
23874,2020.0,TW0009940007,-6.28,-4.19,0.0,-2.09,0.00,3.0,1.0,-0.90
23877,2020.0,JP3423800006,0.00,0.00,0.0,0.00,0.00,0.0,0.0,-1.29
23880,2020.0,ZAE000090692,10.97,-0.38,0.0,0.00,4.07,3.0,1.0,-1.28
23883,2020.0,GB00B4X1RC86,10.47,-0.03,0.0,1.50,0.00,3.0,1.0,-0.01


In [86]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.regression.mixed_linear_model import MixedLM

# Sample DataFrame - replace this with your actual DataFrame
# data = pd.read_csv('your_data.csv')

# Define your model formula
# Include all other features as fixed effects
non_feature_cols = ['isin', 'year', 'ghg_change_real', 'ghg_change_real_next']
target = 'ghg_change_real'
features = '+'.join([f'Q("{col}")' if ' ' in col else col for col in train_data.columns if col not in non_feature_cols])
formula = f'{target} ~ {features}'

# Define the random effects structure
# Here, we're specifying that the random effects vary by 'firm_id'
random_effects = '1 | isin'

# Fit the mixed model
model = MixedLM.from_formula(formula, groups=train_data['isin'], re_formula='1', data=train_data)
result = model.fit()

# Print the summary
print(result.summary())

               Mixed Linear Model Regression Results
Model:               MixedLM   Dependent Variable:   ghg_change_real
No. Observations:    16213     Method:               REML           
No. Groups:          3061      Scale:                28.2694        
Min. group size:     1         Log-Likelihood:       -50986.3278    
Max. group size:     15        Converged:            Yes            
Mean group size:     5.3                                            
--------------------------------------------------------------------
                         Coef.  Std.Err.    z    P>|z| [0.025 0.975]
--------------------------------------------------------------------
Intercept                -0.146    0.092  -1.589 0.112 -0.326  0.034
ghg_change_total          0.100    0.002  40.705 0.000  0.096  0.105
ghg_change_structure     -0.100    0.004 -26.198 0.000 -0.108 -0.093
ghg_change_measure       -0.101    0.003 -33.523 0.000 -0.107 -0.095
ghg_change_othergroup    -0.100    0.003 -32.706 0

In [87]:
# calculate r2 of the fitted model
train_data['predicted_y'] = result.fittedvalues
train_data['residuals'] = result.resid
train_r2 = r2_score(train_data[target], train_data['predicted_y'])
print(train_r2)


0.4330261453030534


In [89]:
# calculate test rmse and r2
test_data['predicted_y'] = result.predict(test_data)
test_rmse = mean_squared_error(test_data[dependent_var], test_data['predicted_y'], squared=False)
test_r2 = r2_score(test_data[dependent_var], test_data['predicted_y'])

In [90]:
print(test_rmse)
print(test_r2)

10.043521677350185
-0.005246539130670191


In [75]:
# calculate mae
test_mae = mean_absolute_error(test_data[dependent_var], test_data['predicted_y'])
print(test_mae)

6.521548290092341
